In [1]:
# Vamos calcular o preço teórico de uma opção de compra de PETR4 utilizando o modelo de Black and Scholes
# Começamos importando as bibliotecas utilizadas de costume para o tratamento de assuntos financeiros

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from pandas_datareader import data as web
%matplotlib inline

# Agora, vamos importar os dados do preço de fechamento diário ajustado da PETR4 por um prazo de 10 anos
# Para isso, opto por utilizar a API Yahoo Finance

acao = 'PETR4.SA'
precos = pd.DataFrame()
precos[acao] = web.DataReader(acao, data_source = 'yahoo', start = '2010-01-01')['Adj Close']
precos.head()

,PETR4.SA
Date,
2010-01-04,31.196165
2010-01-05,30.928682
2010-01-06,31.346632
2010-01-07,31.054062
2010-01-08,30.886881


$$
d_1 = \frac{\ln(\frac{S}{K}) + (r + \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

$$
d_2 = d_1 - s \cdot \sqrt{t} = \frac{\ln(\frac{S}{K}) + (r - \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

In [2]:
# As fórmulas acima correspondem a primeira (d1) e segunda (d2) derivadas da equação de Black and Scholes
# Elas tomam como parâmetros o preço do ativo subjacente (S), Strike (K) da call, taxa de juros (r), volatilidade e tempo.
# Devemos ainda lembrar que, o modelo de Black and Scholes clássico assume:
# (1) Não distribuição de dividendos
# (2) EMH - hipótese do mercado eficiente
# (3) Custos de transação nulos
# (4) Volatilidade e taxa de juros constantes
# Entendido isso, basta definirmos em Python funções iguais às fórmulas acima, o que fica bem simples utilizando funções Numpy

def d1(S, K, r, stdev, T):
    return (np.log(S / K) + (r + stdev ** 2 / 2) * T) / (stdev * np.sqrt(T))
 
def d2(S, K, r, stdev, T):
    return (np.log(S / K) + (r - stdev ** 2 / 2) * T) / (stdev * np.sqrt(T))

$$
\textbf{C} = SN(d_1) - Ke^{-rt}N(d_2) 
$$

In [3]:
# Acima, temos a fórmula do preço da Call tomando como parâmetros a primeira e segunda derivadas
# Além disso, como estamos trabalhando com probabilidades de distribuição, usaremos a função norm.cdf para esse objetivo
# De modo análogo ao que fizemos para d1 e d2, faremos para a função preço, que nesse caso chamei de BSM

def BSM(S, K, r, stdev, T):
        return (S * norm.cdf(d1(S, K, r, stdev, T))) - (K * np.exp(-r * T) * norm.cdf(d2(S, K, r, stdev, T)))

In [5]:
# Agora, basta definirmos o preço do ativo subjacente como o último preço de negociação até a realização desse trabalho
# Usamos a função iloc para obter esse preço

S = precos.iloc[-1]
S

PETR4.SA    19.719999
Name: 2020-11-06 00:00:00, dtype: float64

In [6]:
# Agora, devemos calcular os retornos logarítmicos da série de dados, para posteriormente obtermos o parâmetro stdev

log_returns = np.log(1 + precos.pct_change())
log_returns

,PETR4.SA
Date,
2010-01-04,NaN
2010-01-05,-0.008611
2010-01-06,0.013423
2010-01-07,-0.009377
2010-01-08,-0.005398
...,...
2020-10-30,-0.018311
2020-11-03,0.036801
2020-11-04,0.003556


In [7]:
# Como queremos a vol anual realizada , e não a implícita, anualizaremos stdev

stdev = log_returns.std() * 250 ** 0.5
stdev

PETR4.SA    0.474874
dtype: float64

In [8]:
# Os outros parâmetros podem ser declarados manualmente.
# Considerei a taxa de juros igual à taxa Selic atual (2% a.a)
# Para definir o Strike e o tempo de vida da opção, consultei o opcoes.net para confrontar o preço que iremos obter com o preço de uma opção real
# Escolhi a PETRL220 com último negócio a R$ 0,75, com strike K de R$ 20,61 e 30 dias úteis para o vencimento
# Vamos escrever esses dados

r = 0.02
K = 20.61
T = 30/250

In [9]:
# Nessa etapa, basta chamarmos as funções definidas acima com os parâmetros que delimitados - S, K, r, stdev e T.

d1(S, K, r, stdev, T)

PETR4.SA   -0.171505
dtype: float64

In [10]:
# Analogamente...

d2(S, K, r, stdev, T)

PETR4.SA   -0.336006
dtype: float64

In [11]:
# Por fim, calculamos o preço teórico da Call pelo Método de Black and Scholes

BSM(S, K, r, stdev, T)

PETR4.SA    0.942126
Name: 2020-11-06 00:00:00, dtype: float64

In [12]:
# Ou seja, podemos concluir que o preço de mercado de R$ 0,75 é inferior ao preço teórico de R$ 0,94 calculado acima
# Isso significa que, pela ótica de precificação do modelo, a opção está subvalorizada, já que o preço teórico supera o de mercado
# Isso poderia também ser esperado apenas comparando a volatilidade implícita para a opção 38,4% (no mesmo site) com a volatilidade realizada do período de preços que puxamos 
# A vol implícita é a vol que, na equação de Black and Scholes, iguala o preço de mercado da opção
# Esse estudo NÃO É recomendação de compra nem de venda, apenas analítico e educativo!